# ImbalancedLearningRegression (0.0.1): Usage
---
## SMOGNBoost
Amit Shanbhoug, 8677407 \
Adapted from Nick Kunz's SMOGN package: https://github.com/nickkunz/smogn/blob/master/examples/smogn_example_1_beg.ipynb


## Installation

First, we install ImbalancedLearningRegression from the Github repository. Alternatively, we could install from the official PyPI distribution. However, the developer version is utilized here for the latest release. Works on Kernel: python 3.10.7, there may be some issues if you run with a different version, older or newer.

In [ ]:
%%capture
## suppress install output

## install pypi release
# !pip install ImbalancedLearningRegression

## install developer version
# !pip install git+https://github.com/paobranco/ImbalancedLearningRegression.git

## Dependencies
Next, we load the required dependencies. Here we import `ImbalancedLearningRegression` to later apply Synthetic Minority Over-Sampling Technique for Regression with Gaussian Noise. In addition, we use `pandas` for data handling, and `seaborn` to visualize our results.

In [ ]:
## load dependencies
## load libraries
import numpy as np
import pandas as pd
import sklearn
import math

from sklearn import tree
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import seaborn
import matplotlib.pyplot as plt
from ImbalancedLearningRegression.smogn import smogn
from ImbalancedLearningRegression.smogn_boost import smogn_boost
#from ImbalancedLearningRegression import *
import ImbalancedLearningRegression as iblr
from sklearn.model_selection import train_test_split

# Data Description
The dataset is sourced from Kaggle with a usability score of 10.0 (Source: https://www.kaggle.com/datasets/arashnic/imbalanced-data-practice). This data contains information about  and goal here is to predict whether an individual would be interested in vehicle insurance, denoted by response column "response". The target variable is highly imbalanced - 0: 319594, 1: 62531

## Loading the Data
Below, we load our data (Imbalanced Insurance Data set), In this case, we name our training set `data` and test data as 'test-data'. 


In [ ]:
#ds = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/housing.csv")
#ata = "https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/College.csv"
#test_data = "https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/CollegeTest.csv"
#print(len(ds))

data = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/housing.csv")
test_data = pd.read_csv("https://raw.githubusercontent.com/paobranco/ImbalancedLearningRegression/SMOGNBoost/data/housingTest.csv")

## Introduction to SMOGNBoost
Here we cover the focus of this example. We call the `smogn_boost` function from this package (`ImbalancedLearningRegression.smogn_boost`) and satisfy the minimum required arguments: `data` and `y`.

* data: this argument takes a training data set
* test_data: this argument takes a test data set
* y: this argument takes a string, which specifies a response variable by header name 
* TotalIterations: this argument takes a positive integer, which specifies the total number of iterations
* pert: perturbation / noise percentage
* replace: sampling replacement (bool)
* k: num of neighs for over-sampling (pos int)
* error_threshold: this argument takes a positive integer, which specifies an error threshold 
* rel_thres: user defined relevance threshold 
* samp_method: "balance or extreme" - sampling method is perc

In [ ]:
## conduct smogn
#smogn = smogn(data, y= "SalePrice")

In [ ]:
## conduct smogn_boost
smogn_boost = smogn_boost(data, test_data, y="SalePrice", TotalIterations=1)

## Results
After conducting SMOGNBoost, we briefly examine the results. 






## Conclusion
TO DO

## References

Branco, P., Torgo, L., Ribeiro, R. (2017). SMOGN: A Pre-Processing Approach for Imbalanced Regression. Proceedings of Machine Learning Research, 74:36-50. http://proceedings.mlr.press/v74/branco17a/branco17a.pdf.

Torgo, L., Ribeiro, R. P., Pfahringer, B., & Branco, P. (2013, September). Smote for regression. In Portuguese conference on artificial intelligence (pp. 378-389). Springer, Berlin, Heidelberg. https://researchcommons.waikato.ac.nz/bitstream/handle/10289/8518/smoteR.pdf?sequence=23

Kunz, N. (2019). SMOGN: Synthetic Minority Over-Sampling for Regression with Gaussian Noise (Version 0.1.0). Python Package Index.
https://pypi.org/project/smogn. 

Gareth, J., Daniela, W., Trevor, H., & Robert, T. (2013). An introduction to statistical learning: with applications in R. Spinger.
http://www-bcf.usc.edu/~gareth/ISL/data.html.

